In [1]:
import pandas as pd
import os
cur_dir = os.getcwd()
cur_dir

'/Users/garrettroell/Documents/VS_code_projects/machine_learning_clostridium/lib/6_conditions'

In [2]:
raw_data = pd.read_csv(f'{cur_dir}/data/raw_data.csv')
raw_data.set_index(['composition','trial','time'],drop=True,inplace=True)

smooth_data = pd.read_csv(f'{cur_dir}/data/smooth_data.csv')
smooth_data.set_index(['composition','trial','time'],drop=True,inplace=True)

In [3]:
display(raw_data.head())
display(smooth_data.head())

acetate  biomass  butanol  butyrate  ethanol  \
composition trial time                                                 
1           1     0.58    21.61     0.41     0.04      0.06    10.94   
                  0.65    44.31     0.39     0.05      0.08    15.89   
                  1.02    46.19     0.46     0.21      0.64     8.14   
                  1.67    46.16     0.49     1.18      3.64    10.81   
                  3.70    34.39     0.64     8.44      9.76    20.34   

                        flow rate     H2   CO    CO2  acetate_0  biomass_0  \
composition trial time                                                       
1           1     0.58         20  0.125  0.5  0.375  16.905029   0.429546   
                  0.65         20  0.125  0.5  0.375  16.905029   0.429546   
                  1.02         20  0.125  0.5  0.375  16.905029   0.429546   
                  1.67         20  0.125  0.5  0.375  16.905029   0.429546   
                  3.70         20  0.125  0.5  0.375  16.905029   0.429546   

                        butanol_0  butyrate_0  ethanol_0  
composition trial time                                    
1           1     0.58   0.029818    0.075529   15.89524  
                  0.65   0.029818    0.075529   15.89524  
                  1.02   0.029818    0.075529   15.89524  
                  1.67   0.029818    0.075529   15.89524  
                  3.70   0.029818    0.075529   15.89524

CO    CO2     H2    acetate   biomass   butanol  \
composition trial time                                                     
1           1     0.0   0.5  0.375  0.125  16.905029  0.429546  0.029818   
                  0.1   0.5  0.375  0.125  18.001195  0.421076  0.028260   
                  0.2   0.5  0.375  0.125  19.831073  0.410803  0.028316   
                  0.3   0.5  0.375  0.125  22.196249  0.403602  0.031357   
                  0.4   0.5  0.375  0.125  25.293925  0.403709  0.028700   

                        butyrate    ethanol  flow rate  acetate_0  biomass_0  \
composition trial time                                                         
1           1     0.0   0.075529  15.895240       20.0  16.905029   0.429546   
                  0.1   0.061033  15.607023       20.0  16.905029   0.429546   
                  0.2   0.051013  15.144135       20.0  16.905029   0.429546   
                  0.3   0.050367  14.522308       20.0  16.905029   0.429546   
                  0.4   0.035514  13.956015       20.0  16.905029   0.429546   

                        butanol_0  butyrate_0  ethanol_0  
composition trial time                                    
1           1     0.0    0.029818    0.075529   15.89524  
                  0.1    0.029818    0.075529   15.89524  
                  0.2    0.029818    0.075529   15.89524  
                  0.3    0.029818    0.075529   15.89524  
                  0.4    0.029818    0.075529   15.89524

In [4]:
from machine_learning.get_trained_models import get_trained_models

trained_models = get_trained_models(raw_data, smooth_data)

raw
gradient boosting
raw, gradient boosting, comp excluded from training: none
raw, gradient boosting, comp excluded from training: 1
raw, gradient boosting, comp excluded from training: 2
raw, gradient boosting, comp excluded from training: 3
raw, gradient boosting, comp excluded from training: 4
raw, gradient boosting, comp excluded from training: 5
raw, gradient boosting, comp excluded from training: 6
random forest
raw, random forest, comp excluded from training: none
raw, random forest, comp excluded from training: 1
raw, random forest, comp excluded from training: 2
raw, random forest, comp excluded from training: 3
raw, random forest, comp excluded from training: 4
raw, random forest, comp excluded from training: 5
raw, random forest, comp excluded from training: 6
support vector
raw, support vector, comp excluded from training: none
raw, support vector, comp excluded from training: 1
raw, support vector, comp excluded from training: 2
raw, support vector, comp excluded from tr

In [9]:
from machine_learning.get_model_predictions import get_model_predictions
measured_time_predictions = get_model_predictions(trained_models, raw_data)
smoothed_time_predictions = get_model_predictions(trained_models, smooth_data)

In [10]:
from machine_learning.get_model_metrics import get_pearson_r2, get_rmse

def evaluate_models(model_dict, measured_values, metric):
    species_set = ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']
    test_comp_set=[1,2,3,4,5,6]
    index_set = ['gradient boosting, raw', 'random forest, raw', 'support vector, raw', 'neural net (relu), raw', 'neural net (tanh), raw', 'neural net (log), raw', 'lasso, raw', 'gradient boosting, smooth', 'random forest, smooth', 'support vector, smooth', 'neural net (relu), smooth', 'neural net (tanh), smooth', 'neural net (log), smooth',  'lasso, smooth']
    
    for species in species_set:
        data = {}
        for test_comp in test_comp_set:
            data[test_comp] = []
            for model_name in model_dict.keys():
                if str(test_comp) in model_name:
                    predicted_species_values = list(measured_time_predictions[model_name].loc[test_comp][species])
                    measured_species_values = list(raw_data.loc[test_comp][species])

                    r2 = get_pearson_r2(measured_species_values, predicted_species_values)
                    rmse = get_rmse (measured_species_values, predicted_species_values)
                    if metric == 'r2':
                        data[test_comp].append(r2)
                    elif metric == 'rmse':
                        data[test_comp].append(rmse)
                    else:
                        print('unknown metric')
        species_data = pd.DataFrame.from_dict(data)
        species_data[f'model for {species}'] = index_set
        species_data.set_index(f'model for {species}', inplace=True, drop=True)
        display(species_data)

In [11]:
evaluate_models(trained_models, measured_time_predictions, 'r2')

,1,2,3,4,5,6
model for acetate,,,,,,
"gradient boosting, raw",2.958011e-01,7.764942e-01,0.617148,0.191956,0.856450,0.302761
"random forest, raw",2.522518e-01,7.644307e-01,0.587654,0.157867,0.798625,0.043661
"support vector, raw",5.720648e-02,3.716270e-02,0.081652,0.004536,0.846740,0.960345
"neural net (relu), raw",2.073528e-01,1.133004e-01,0.000893,0.011260,0.651165,0.952635
"neural net (tanh), raw",2.123190e-01,1.237394e-01,0.000255,0.003001,0.817144,0.994335
"neural net (log), raw",4.913926e-03,1.712014e-01,0.001701,0.003224,0.722328,0.979667
"lasso, raw",1.840623e-01,1.432170e-32,0.007425,0.000000,0.000000,0.000000
"gradient boosting, smooth",1.297973e-01,6.194582e-01,0.272389,0.040963,0.923579,0.033474
"random forest, smooth",9.302624e-02,5.582618e-01,0.517924,0.228849,0.634781,0.475678


,1,2,3,4,5,6
model for biomass,,,,,,
"gradient boosting, raw",0.642858,0.785823,0.869391,0.529853,0.757601,0.020678
"random forest, raw",0.651467,0.868636,0.935633,0.301259,0.758663,0.337224
"support vector, raw",0.879139,0.810641,0.403790,0.111539,0.000367,0.583200
"neural net (relu), raw",0.488194,0.524136,0.155311,0.351885,0.000367,0.550094
"neural net (tanh), raw",0.417693,0.790150,0.006561,0.004362,0.000304,0.624143
"neural net (log), raw",0.178415,0.012591,0.044679,0.326442,0.004824,0.564801
"lasso, raw",0.888325,0.832773,0.728352,0.240102,0.000367,0.583200
"gradient boosting, smooth",0.644230,0.579270,0.868062,0.283907,0.341246,0.913756
"random forest, smooth",0.673498,0.668212,0.861036,0.441194,0.368675,0.177981


,1,2,3,4,5,6
model for butanol,,,,,,
"gradient boosting, raw",0.928963,0.737668,0.884092,0.343854,0.897930,0.114149
"random forest, raw",0.788628,0.736824,0.930535,0.397676,0.912236,0.971014
"support vector, raw",0.932921,0.747477,0.859992,0.596095,0.961781,0.892565
"neural net (relu), raw",0.801231,0.405991,0.973704,0.468829,0.997097,0.868700
"neural net (tanh), raw",0.312777,0.573523,0.946372,0.675728,0.989586,0.926748
"neural net (log), raw",0.917115,0.689742,0.686709,0.641761,0.963483,0.863142
"lasso, raw",0.908492,0.747119,0.827766,0.600728,0.961781,0.892565
"gradient boosting, smooth",0.845375,0.351020,0.930090,0.472038,0.987876,0.641305
"random forest, smooth",0.828108,0.854819,0.945625,0.434475,0.984827,0.704063


,1,2,3,4,5,6
model for butyrate,,,,,,
"gradient boosting, raw",0.543996,0.548466,0.057217,0.250711,0.487558,0.869528
"random forest, raw",0.473715,0.627356,0.294213,0.353087,0.671834,0.900298
"support vector, raw",0.310146,0.840786,0.892178,0.150992,0.551598,0.848379
"neural net (relu), raw",0.195353,0.877256,0.761024,0.239150,0.551598,0.740008
"neural net (tanh), raw",0.076851,0.942590,0.867515,0.327931,0.899131,0.950518
"neural net (log), raw",0.096867,0.161868,0.003731,0.264450,0.560670,0.783207
"lasso, raw",0.421602,0.881000,0.819901,0.166175,0.551598,0.775289
"gradient boosting, smooth",0.458294,0.740297,0.050771,0.342683,0.919779,0.161061
"random forest, smooth",0.391175,0.709590,0.056135,0.342444,0.862226,0.299330


,1,2,3,4,5,6
model for ethanol,,,,,,
"gradient boosting, raw",0.830771,0.878522,0.773058,0.407522,0.416628,0.568748
"random forest, raw",0.770415,0.924678,0.887105,0.380812,0.532996,0.349571
"support vector, raw",0.461793,0.816614,0.873767,0.851661,0.359281,0.467208
"neural net (relu), raw",0.206273,0.831036,0.872578,0.555822,0.359281,0.512833
"neural net (tanh), raw",0.761642,0.312036,0.228059,0.687160,0.194169,0.285163
"neural net (log), raw",0.160565,0.585535,0.294739,0.663316,0.289484,0.387592
"lasso, raw",0.401092,0.824191,0.826974,0.859837,0.359281,0.467208
"gradient boosting, smooth",0.733558,0.944394,0.522880,0.373147,0.922775,0.529333
"random forest, smooth",0.733469,0.953704,0.079022,0.287022,0.812607,0.447379


In [8]:
evaluate_models(trained_models, measured_time_predictions, 'rmse')

,1,2,3,4,5,6
model for acetate,,,,,,
"gradient boosting, raw",12.422380,10.194891,20.646782,18.490849,8.526668,14.554607
"random forest, raw",12.775860,10.586625,22.854629,17.546810,10.172667,16.769244
"support vector, raw",23.576583,25.483475,30.118801,20.497563,10.751442,13.686299
"neural net (relu), raw",25.760376,22.338226,32.524701,16.980186,13.315539,16.895909
"neural net (tanh), raw",18.648657,36.006192,41.121218,10.456931,15.222851,19.317701
"neural net (log), raw",24.087654,42.059790,46.674680,14.048685,21.072834,25.074319
"lasso, raw",17.993950,23.907623,27.208792,17.095258,11.005581,14.558802
"gradient boosting, smooth",13.697182,15.430740,29.799038,15.299753,6.305459,18.447537
"random forest, smooth",14.319925,15.988466,21.076560,12.160897,7.226960,20.942217


,1,2,3,4,5,6
model for biomass,,,,,,
"gradient boosting, raw",0.090302,0.081637,0.078746,0.075973,0.047176,0.054967
"random forest, raw",0.098726,0.082687,0.084682,0.078048,0.037382,0.054677
"support vector, raw",0.114066,0.163125,0.077755,0.078560,0.091731,0.104156
"neural net (relu), raw",0.175361,0.124934,0.113975,0.233461,0.315002,0.229501
"neural net (tanh), raw",0.278280,0.143396,0.103292,0.077561,0.110229,0.038227
"neural net (log), raw",0.149603,0.141983,0.088338,0.072278,0.092782,0.098997
"lasso, raw",0.115107,0.092836,0.102330,0.102248,0.089072,0.105352
"gradient boosting, smooth",0.110002,0.101863,0.104518,0.065768,0.040194,0.058280
"random forest, smooth",0.069707,0.100605,0.086512,0.067010,0.024274,0.043409


,1,2,3,4,5,6
model for butanol,,,,,,
"gradient boosting, raw",6.535660,4.216003,4.777398,4.156039,2.917200,2.877121
"random forest, raw",5.407436,3.286717,3.756009,4.163691,4.312563,2.727335
"support vector, raw",7.343009,3.972999,3.204006,4.104601,0.873154,0.813144
"neural net (relu), raw",8.325654,5.405552,1.083793,4.026061,2.415200,4.586122
"neural net (tanh), raw",8.304097,3.586383,2.493649,2.916944,2.540219,1.263752
"neural net (log), raw",9.633715,5.613819,3.748216,4.729530,3.545623,2.578260
"lasso, raw",5.650842,3.230860,2.849106,3.258065,3.369770,3.531981
"gradient boosting, smooth",7.287548,4.765948,4.692540,3.813159,3.325666,2.545267
"random forest, smooth",5.243444,2.442482,5.402858,4.150687,4.613314,1.288534


,1,2,3,4,5,6
model for butyrate,,,,,,
"gradient boosting, raw",3.445155,2.794059,3.792565,4.736045,5.337285,6.311998
"random forest, raw",3.788093,2.806764,2.688859,4.990703,3.871213,6.864297
"support vector, raw",4.288005,3.719686,2.123109,4.605010,5.381258,7.479564
"neural net (relu), raw",5.744438,4.440088,3.893550,4.606655,5.093759,4.922752
"neural net (tanh), raw",5.750057,3.988586,3.514592,4.522696,5.503285,6.332530
"neural net (log), raw",5.476539,4.543848,2.566788,2.632660,7.732450,8.363275
"lasso, raw",3.704553,3.530819,2.454428,4.992723,3.481963,4.161202
"gradient boosting, smooth",4.079146,2.875142,3.123560,5.006414,3.942726,7.160881
"random forest, smooth",4.307165,2.173693,3.617371,5.493177,1.814884,7.777315


,1,2,3,4,5,6
model for ethanol,,,,,,
"gradient boosting, raw",34.689376,12.092789,26.281941,32.311467,26.054644,46.664533
"random forest, raw",29.224420,5.676490,20.414097,31.344850,25.044884,40.806524
"support vector, raw",28.093724,13.056413,37.874582,26.991887,10.013022,15.756283
"neural net (relu), raw",35.326621,10.260096,30.076385,32.328403,10.610101,24.068812
"neural net (tanh), raw",10.251984,20.643420,39.847513,30.320713,18.624853,17.774324
"neural net (log), raw",7.890204,24.574801,43.418721,30.762188,12.223728,11.572147
"lasso, raw",42.313048,11.954520,28.531754,30.137458,26.830993,30.426170
"gradient boosting, smooth",34.647512,11.340595,29.217187,32.639334,1.620649,39.525206
"random forest, smooth",29.998015,5.577264,35.064750,35.893904,6.588141,42.755354
